In [1]:
# Dependencies
import requests
import json
import pprint
from pymongo import MongoClient


In [2]:
# request to get JSON that contains all team ids
teams_response = requests.get('https://statsapi.web.nhl.com/api/v1/teams').json()

In [3]:
# pretty print teams_response
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(teams_response)

{   'copyright': 'NHL and the NHL Shield are registered trademarks of the '
                 'National Hockey League. NHL and NHL team marks are the '
                 'property of the NHL and its teams. © NHL 2018. All Rights '
                 'Reserved.',
    'teams': [   {   'abbreviation': 'NJD',
                     'active': True,
                     'conference': {   'id': 6,
                                       'link': '/api/v1/conferences/6',
                                       'name': 'Eastern'},
                     'division': {   'abbreviation': 'M',
                                     'id': 18,
                                     'link': '/api/v1/divisions/18',
                                     'name': 'Metropolitan',
                                     'nameShort': 'Metro'},
                     'firstYearOfPlay': '1982',
                     'franchise': {   'franchiseId': 23,
                                      'link': '/api/v1/franchises/23',
          

                                      'link': '/api/v1/franchises/25',
                                      'teamName': 'Oilers'},
                     'franchiseId': 25,
                     'id': 22,
                     'link': '/api/v1/teams/22',
                     'locationName': 'Edmonton',
                     'name': 'Edmonton Oilers',
                     'officialSiteUrl': 'http://www.edmontonoilers.com/',
                     'shortName': 'Edmonton',
                     'teamName': 'Oilers',
                     'venue': {   'city': 'Edmonton',
                                  'id': 5100,
                                  'link': '/api/v1/venues/5100',
                                  'name': 'Rogers Place',
                                  'timeZone': {   'id': 'America/Edmonton',
                                                  'offset': -7,
                                                  'tz': 'MST'}}},
                 {   'abbreviation': 'VAN',
               

In [4]:
# Number of teams in the NHL
len(teams_response['teams'])

31

In [11]:
# Collects the ids of active NHL teams

team_ids = []

for x in range(len(teams_response['teams'])):
    team_ids.append(teams_response['teams'][x]['id'])

In [13]:
# Uses the collected ids to loop through, pull stats & write to MongoDB

for team in team_ids:
    
    query_url = f"https://statsapi.web.nhl.com/api/v1/teams/{team}/stats"
    team_stats = requests.get(query_url).json()
    
    client = MongoClient('localhost', 27017)
    db = client['nhl-database']
    collection = db['STATS']
    collection.insert_one(team_stats)
    

In [7]:
import pandas as pd
from pandas.io.json import json_normalize

In [8]:
datapoints = pd.DataFrame(list(collection.find({})))

In [9]:
pre_df_data = []

for i in range(len(team_ids)):
    pre_df_data.append((datapoints.stats[i][0]['splits'][0]))

In [10]:
stats_df = json_normalize(pre_df_data)
stats_df

,stat.evGGARatio,stat.faceOffWinPercentage,stat.faceOffsLost,stat.faceOffsTaken,stat.faceOffsWon,stat.gamesPlayed,stat.goalsAgainstPerGame,stat.goalsPerGame,stat.losses,stat.ot,...,stat.winLeadFirstPer,stat.winLeadSecondPer,stat.winOppScoreFirst,stat.winOutshootOpp,stat.winOutshotByOpp,stat.winScoreFirst,stat.wins,team.id,team.link,team.name
0,0.8929,46.7,885.0,1661.0,776.0,26,3.500,2.885,12,5,...,0.600,0.800,0.100,0.400,0.375,0.500,9,1,/api/v1/teams/1,New Jersey Devils
1,1.2558,47.8,739.0,1415.0,676.0,25,2.800,3.000,9,3,...,0.545,0.846,0.400,0.143,0.706,0.600,13,2,/api/v1/teams/2,New York Islanders
2,0.8750,48.7,819.0,1596.0,777.0,28,3.107,2.714,12,3,...,0.400,0.714,0.462,0.500,0.444,0.467,13,3,/api/v1/teams/3,New York Rangers
3,0.9455,55.9,631.0,1430.0,799.0,25,3.520,3.000,12,2,...,0.778,0.818,0.313,0.267,0.571,0.667,11,4,/api/v1/teams/4,Philadelphia Flyers
4,1.0690,49.7,789.0,1568.0,779.0,25,3.280,3.320,10,5,...,0.500,0.900,0.333,0.364,0.455,0.438,10,5,/api/v1/teams/5,Pittsburgh Penguins
5,1.2353,48.7,797.0,1554.0,757.0,26,2.462,2.692,8,4,...,0.818,0.909,0.357,0.455,0.571,0.750,14,6,/api/v1/teams/6,Boston Bruins
6,1.1224,47.3,895.0,1697.0,802.0,28,2.786,2.964,8,3,...,0.833,0.800,0.385,0.545,0.625,0.800,17,7,/api/v1/teams/7,Buffalo Sabres
7,1.0690,47.1,892.0,1685.0,793.0,27,3.259,3.000,10,5,...,0.750,0.667,0.308,0.467,0.417,0.571,12,8,/api/v1/teams/8,Montréal Canadiens
8,0.9306,50.1,864.0,1733.0,869.0,27,4.037,3.630,12,3,...,0.583,0.700,0.385,0.625,0.368,0.500,12,9,/api/v1/teams/9,Ottawa Senators
9,1.4130,51.6,805.0,1662.0,857.0,27,2.593,3.630,8,0,...,1.000,1.000,0.429,0.636,0.800,1.000,19,10,/api/v1/teams/10,Toronto Maple Leafs
